In [1]:
import pandas as pd

macro_data = {
    "YearQuarter": ["2014 Q3", "2014 Q4", "2015 Q1", "2015 Q2", "2015 Q3"],
    "bank_rate":   [0.5,       0.5,       0.5,       0.5,       0.5]  # example
}
macro_df = pd.DataFrame(macro_data)
macro_df.head()


,YearQuarter,bank_rate
0,2014 Q3,0.5
1,2014 Q4,0.5
2,2015 Q1,0.5
3,2015 Q2,0.5
4,2015 Q3,0.5


In [2]:
macro_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   YearQuarter  5 non-null      object 
 1   bank_rate    5 non-null      float64
dtypes: float64(1), object(1)
memory usage: 212.0+ bytes


In [3]:
df_dvla = pd.read_csv("/content/dvla_with_lags_long.csv")

df_dvla_macro = df_dvla.merge(
    macro_df,
    on="YearQuarter",
    how="left"
)

df_dvla_macro.head()

,BodyType,Make,GenModel,Model,Fuel,YearQuarter,Registrations,Date,Registrations_lag_1,Registrations_lag_2,Registrations_lag_4,bank_rate
0,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,2014 Q3,0,2014-09-30,NaN,NaN,NaN,0.5
1,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,2014 Q4,0,2014-12-31,0.0,NaN,NaN,0.5
2,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,2015 Q1,0,2015-03-31,0.0,0.0,NaN,0.5
3,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,2015 Q2,0,2015-06-30,0.0,0.0,NaN,0.5
4,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,2015 Q3,0,2015-09-30,0.0,0.0,0.0,0.5


In [4]:
feature_cols = [
    "Registrations_lag_1",
    "Registrations_lag_2",
    "Registrations_lag_4",
    "bank_rate"          # new macro feature
]

df_model = df_dvla_macro.dropna(subset=feature_cols).copy()
X = df_model[feature_cols]
y = df_model["Registrations"]

In [5]:
ts_df = df_dvla.groupby("YearQuarter", as_index=False)["Registrations"].sum()
ts_df = ts_df.sort_values("YearQuarter")


In [6]:
# convert YearQuarter to a date (e.g. last day of quarter)
ts_df["ds"] = pd.to_datetime(ts_df["YearQuarter"].str.replace(" Q1","-03-31")
                                                      .str.replace(" Q2","-06-30")
                                                      .str.replace(" Q3","-09-30")
                                                      .str.replace(" Q4","-12-31"))
ts_df["y"] = ts_df["Registrations"]


In [7]:
from prophet import Prophet

m = Prophet()
m.fit(ts_df[["ds", "y"]])

future = m.make_future_dataframe(periods=4, freq="Q")  # next 4 quarters
forecast = m.predict(future)


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1875: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  dates = pd.date_range(
